In [75]:
import pandas as pd
from alphabet_detector import AlphabetDetector
import string
import re
import nltk
import numpy as np
import operator
from nltk.stem.snowball import SnowballStemmer

# Data cleaning

In [2]:
path_to_data = '../data/comments1.csv'

In [3]:
df = pd.read_csv(path_to_data)
df = df.dropna()

In [4]:
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

In [5]:
def clean(x):
    ad = AlphabetDetector()
    res = x
    for ch in string.punctuation:                                                                                                     
        res = res.replace(ch, ' ')
    res = ''.join([i for i in res if not i.isdigit()])
    res = res.lower()
    res = emoji_pattern.sub(r' ', res)
    res = res.replace('\n', ' ')
    res = res.replace('\t', ' ')
    res = res.replace('\ufeff', ' ')
    res = res.replace('\r\n', '  ')
    res = res.replace('\xa0', ' ')
    res = res.replace('ё', 'е')
    res = re.sub(' +',' ', res)
    if  not ad.only_alphabet_chars(res, "CYRILLIC"): 
        res = ''
    return res

In [6]:
df.text = df.text.apply(clean)
df = df[df.text != '']

In [7]:
#df.to_csv('../data/comments_clean1.csv', index=False)

In [8]:
#remove comments that were manualy labeld
manual = pd.read_csv('../data/manual.csv')
df = df[~df.id.isin(manual.id)]

# Stemming bad words

In [43]:
def stem(word):
    stemmer = SnowballStemmer("russian", ignore_stopwords=True) 
    stemmed_word = stemmer.stem(word)
    if len(stemmed_word) <= 2:
        return word
    return stemmed_word

In [76]:
path_to_bad_words = '../data/bad_words.txt'

In [77]:
bad_words = open(path_to_bad_words).read().split('\n')[:-1]

In [78]:
report_dictionary = [] # array that will contain all bad words
stemmed_dictionary = [] # array that will contain actual stems used for finding bad words

In [79]:
for word in bad_words:
    report_dictionary.append(word)
    stemmed_dictionary.append(stem(word))

In [80]:
stemmed_dictionary = [i for i in stemmed_dictionary if len(i) > 2] 

In [81]:
stemmed_dictionary = list(set(stemmed_dictionary))

In [82]:
stemmed_dictionary.append('хуй') # nltk stemmer can not corectly stem word хуй
stemmed_dictionary.append('хуе')

# label coments with bad words

In [83]:
def label(x):
    global stemmed_dictionary
    tokens = nltk.word_tokenize(x)
    for bad_word in stemmed_dictionary:
        for token in tokens:
            if bad_word in token:
                return True
    return False

In [84]:
df['label'] = df.text.apply(label)

# Iterative process of finding new bad words

## Word stemming and counting

In [28]:
all_stemmed = dict()
for sentence in df.text:
    for token in nltk.word_tokenize(sentence):
        if token not in all_stemmed:
            all_stemmed[token] = stem(token)

In [29]:
all_counts = dict()
for sentence in df.text:
    tokens = nltk.word_tokenize(sentence)
    for token in tokens:
        word = stem(token)
        if word in all_counts:
            all_counts[word] += 1
        else:
            all_counts[word] = 1

# Likelihood of words being in comments labeld as bad or labeld as not bad

In [30]:
def likelihood(label):
    labeld_part = dict()
    part = df[df.label == label]
    for index, row in part.iterrows():
        sentence = row.text
        tokens = nltk.word_tokenize(sentence)
        for token in tokens:
            stemed = stem(token)
            if stemed != '':
                if all_counts[stemed] >= 20:
                    if stemed in labeld_part:
                        labeld_part[stemed] += 1.0/len(part)
                    else:
                        labeld_part[stemed] = 1.0/len(part)
    return labeld_part

## Criteria functions for decision if word is bad or not bad

In [31]:
def relative_distance(p_good, p_bad):
    difference = dict()
    for key in p_good:
        if key in p_bad:
            difference[key] =(p_bad[key] - p_good[key])/ np.maximum( p_bad[key], p_good[key]) 
    return difference

In [32]:
def logg_odds(p_good, p_bad):
    ratio = dict()
    for key in p_good:
        if key in p_bad:
            odds_good =  p_good[key]/(1 -  p_good[key])
            odds_bad = p_bad[key]/(1- p_bad[key])
            ratio[key] = np.log(odds_bad/odds_good)
    return ratio

## Pointwise mutual information

In [72]:
def make_pairs(df_bad):
    pairs = dict()
    for sentence in df_bad.text:
        selection = []
        for token in nltk.word_tokenize(sentence):
            word = stem(token)
            if word in stemmed_dictionary and all_counts[word] >= 20:
                selection.append(word)
        for bad_word in selection:
            for token in nltk.word_tokenize(sentence):
                word = stem(token)
                if bad_word != word and word != '' and all_counts[word] >= 20:
                    key = bad_word + '-' + word
                    if key in pairs:
                        pairs[key] += 1
                    else:
                        pairs[key] = 1
    return pairs

In [73]:
def pmi():
    bad_likelihood = likelihood(True)
    pairs = make_pairs(df[df.label == True])
    result = dict()
    for key in pairs:
        words = key.split('-')
        pxy = float(pairs[key]/len(df[df.label == True]))
        px = bad_likelihood[words[0]]
        py = bad_likelihood[words[1]]
        result[key] = np.log(pxy/(px*py))
    return result
        
        
        

In [74]:
sort(pmi())

[('еба-отдад', 3.68998994839796),
 ('бля-проведен', 3.613028907261832),
 ('бля-марк', 3.6130289072618313),
 ('сук-шлем', 3.4741619797141685),
 ('пизд-проведен', 3.4270560456353834),
 ('пизд-марк', 3.4270560456353834),
 ('хуй-деся', 3.2199863191522304),
 ('бля-мут', 3.207563799153667),
 ('пизд-позитивн', 3.021590937527219),
 ('пизд-поработа', 3.021590937527219),
 ('пизд-поуч', 3.021590937527219),
 ('пизд-благополуч', 3.021590937527219),
 ('пизд-снг', 3.021590937527219),
 ('пизд-мут', 3.0215909375272187),
 ('бля-тим', 3.0199651852588687),
 ('еба-дружищ', 2.996842767838015),
 ('еба-увелич', 2.996842767838015),
 ('еба-тактик', 2.996842767838015),
 ('еба-оборот', 2.996842767838015),
 ('еба-проведен', 2.996842767838015),
 ('еба-концлагер', 2.996842767838015),
 ('еба-марк', 2.9968427678380145),
 ('сук-людск', 2.9633363559481776),
 ('бля-позитивн', 2.9198817267018864),
 ('бля-либерал', 2.9198817267018864),
 ('бля-тупоголов', 2.9198817267018864),
 ('бля-заинтересова', 2.9198817267018864),
 ('бл

## Detecting new bad words

In [33]:
def unstem(stem):
    res = []
    for key in all_stemmed:
        if all_stemmed[key] == stem:
            res.append(key)
    return res    

In [34]:
def sort(x, rev = True):
    return sorted(x.items(), key=operator.itemgetter(1), reverse=rev)

In [35]:
def update_dictionary(word):
    global stemmed_dictionary
    if len(word) <= 2:
        return False
    #for key in stemmed_dictionary:
        #if word in key:
            #return False
    stemmed_dictionary.append(word)
    return True

In [85]:
def iterate(df, report_dictionary, stemmed_dictionary):
    iter = True
    while(iter):
        print('-----------------------------------')
        print('labeld as abusive: ', len(df[df.label == True]))
        #print('List of bad_words')
        #print(report_dictionary)
        p_bad = likelihood(True)
        p_good = likelihood(False)
        prob = logg_odds(p_good, p_bad)
        print(sort(prob)[:10])
        print('New bad words')
        new_words = []
        for key in prob:
            if prob[key] > 3:
                if update_dictionary(key):
                    new_words += unstem(key)
                    print( unstem(key))
        report_dictionary += new_words
        if len(new_words) == 0:
            iter = False
            break
        #print('\nNew dictionary')
        #print(report_dictionary)
        df['label'] =  df.text.apply(label)
        print('relabeld as abusive: ', len(df[df.label == True]))

In [86]:
iterate(df, report_dictionary, stemmed_dictionary)

-----------------------------------
labeld as abusive:  13139


/home/bandrusyak/.local/lib/python3.5/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in log
  import sys


[('чао', 3.9378342882185584), ('засад', 3.888967895821777), ('закупа', 3.291252155597222), ('колхоз', 3.0902765588445003), ('иностранц', 2.9416736877701126), ('овец', 2.73892368590585), ('звон', 2.7261046771483075), ('инфраструктур', 2.6847802480279386), ('хохлятск', 2.6275456233888828), ('тип', 2.501735187796267)]
New bad words
['закупаются', 'закупаем', 'закупает', 'закупать', 'закупается', 'закупают']
['чао']
['колхозы', 'колхозу', 'колхозах', 'колхоз', 'колхоза', 'колхозов', 'колхозам']
['засада', 'засаде', 'засадят', 'засадили', 'засадить', 'засаду', 'засади']
relabeld as abusive:  13150
-----------------------------------
labeld as abusive:  13150
[('иностранц', 2.9404915527277984), ('овец', 2.7377423284789804), ('звон', 2.7249229257320584), ('инфраструктур', 2.6835989544462264), ('хохлятск', 2.6263643936426013), ('тип', 2.500535284087174), ('чувствова', 2.4500290416311175), ('наркоман', 2.432952668770567), ('дыр', 2.3965088196082287), ('алкогол', 2.358692292440676)]
New bad word

# Evaluation

In [88]:
manual['evaluation'] = manual.text.apply(label)

In [89]:
tp = 0
tn = 0
fp = 0
fn = 0
for index, row in manual.iterrows():
    if row.label == True and row.evaluation == True:
        tp += 1
    if row.label == False and row.evaluation == False:
        tn += 1
    if row.label == False and row.evaluation == True:
        fp += 1
    if row.label == True and row.evaluation == False:
        #print(row.text)
        fn += 1
accuracy = (tp + fp)/(tp + fp + fn + fp)
precision = tp/(tp + fp)
recall = tp/(tp + fn)
f1  = 2 * (precision * recall)/(precision + recall)
print('accuracy: ', accuracy)
print('precision: ', precision)
print('recall: ', recall)
print('f1: ', f1)

accuracy:  0.6260330578512396
precision:  0.7722772277227723
recall:  0.6763005780346821
f1:  0.7211093990755009
